# Lorentz transformation
In this lesson we will write and use functions for working with 4-vectors and Lorentz transformations.

## Pre-requisites
First we need to load the modules we need. We can also check whether loading was succeful and the version of the imported module.

In [19]:
import astropy
astropy.__version__

'4.3.1'

numpy is typically imported as np:

In [20]:
import numpy as np
np.__version__

'1.20.3'

We will use matplotlib for plotting:

In [21]:
import matplotlib
matplotlib.__version__

'3.4.3'

From the astropy module we need only the units and the constants, we will import them as u and c.

In [22]:
from astropy import units as u
from astropy import constants as c

We will now import the pylorentz module, which will allow us to define a general 4-vector class followed by 4-momentum and 4-position subclasses.

In [23]:
from pylorentz import Position4, Momentum4

### Time dilation
Let's use the 4-position vectors to examine time dilation.

In [33]:
distance = 1
beta = 0.9
gamma = (1 - beta**2)**(-0.5)
print, gamma

Delta_t = 2 * distance

Event0 = Position4(0,0,0,0)
Event1 = Position4(0.5 * Delta_t, 0, distance, 0)
Event2 = Position4(Delta_t, 0, 0, 0)
print(repr(Event0))
print(repr(Event1))
print(repr(Event2))

Event0_prime = Event0.boost(1,0,0,beta)
print(repr(Event0_prime))
Event1_prime = Event1.boost(1,0,0,beta)
print(repr(Event1_prime))
Event2_prime = Event2.boost(1,0,0,beta)
print(repr(Event2_prime))

Delta_t_prime = Event2_prime[0] - Event0_prime[0]
print(Delta_t_prime)
print(Delta_t_prime / Delta_t)
print(gamma)

Position4(0, 0, 0, 0)
Position4(1, 0, 1, 0)
Position4(2, 0, 0, 0)
Position4(0, 0, 0, 0)
Position4(2.29416, -2.06474, 1, 0)
Position4(4.58831, -4.12948, 0, 0)
4.588314677411236
2.294157338705618
2.294157338705618


### Velocity addition

In [39]:
v1 = 0.5
gamma1 = (1-v1*v1)**(-0.5)
v2 = 0.3
gamma2 = (1-v2*v2)**(-0.5)

p1 = Momentum4(gamma1,gamma1*v1,0,0)
p3 = p1.boost(-1,0,0,beta=v2)

print(p3[1]/p3[0])
print,(v1+v2)/(1+v1*v2)

0.8386278693775345


(<function print>, 0.6956521739130436)

## Particle Distributions (photons, electrons, protons)
In this course we will not discuss photons of a single energy, but particle distributions of photons, electrons or protons. We will study the particles over several orders of magnitude in energy, let's say from 1 MeV (10⁶ eV) to 100 TeV (10¹⁴ eV). We will create a log space with 15 data points:

We study non-thermal emission, so our photons will not follow a block-body spectrum but typically a power law. A power law has the following form:
$$ F(E) = A \times \left( \frac{E}{E_0} \right)^{-\Gamma} $$
$A$ is the amplitude, i.e. the function's value at the reference energy $E_0$. $\Gamma$ is called the spectral index. Note the minus sign in the exponent, so $\Gamma$ is typically positive. We often use 1 TeV as reference energy:

Let's define some example parameters:

A power law is already defined in astropy. Let's make use of it.

Now we can call this function on our energy array defined above:

### Plotting
We will use pyplot from matplotlib to do some plotting. pyplot is usually imported as plt:

Let's make a plot of our y value over our energy array:

This looks a bit strange. Keep in mind that our energies span 14 orders of magnitude. Let's try to get the energy axis in log scale:

Looks better already. But is our function really zero above an energy of 10^7 eV? Let's try a log-log plot:

Looks good. We see that a power law is represented as a straight line in a log-log plot. Now let's add titles to the axes and a legend:

You see that the y axis is in units TeV⁻¹. It is a differential particle number, how many particles are found in an infinitesimal small bin of with $dE$:
$$\frac{dN}{dE}(E).$$
If you want to know how many particles are in the entire particle spectrum then you have to integrate over it:
$$ N = \int_{E_1}^{E_2} \frac{dN}{dE}(E) dE.$$
For a power law it is easy to do the integration analytically. But it may be difficult for more complicated spectra. We can use numpy for a numerical integration.

There are no units as this is simply a particle number. Keep in mind that the trapezoidal rule is not very accurate and depends on the number of points. Having more points would increase the accuracy. Other, more sophisticated integration modules exist for python.

Similarly we can calculate the total energy in the spectrum. We need to integrate over all energies:
$$ E_{tot} = \int_{E_1}^{E_2} E \frac{dN}{dE}(E) dE.$$
We can achieve this by multiplying the $y$ array with the energies array and then integrate.

### Exercise
Make a plot of three power laws with indices 1.5, 2.0 and 2.5. Keep the amplitude as above. Do not forget the title on the x-axis and the legend. Calculate the number of particles and the total energy in each spectrum.

**[3 marks]**

## Cyclotron Frequency

Here are some values fo an example. We will use the geo-magnetic field:

The electron has one elementary charge, we do not care about the sign here:

Let's put everything together:

Depending on the system you are using the elementary charge has different values and units. So we have to be specific here. All calculations in this course are in standard units, so let's try this.

How many cycles per second? We have to devide by the angle of one full circle, $2\pi$.

### Exercise

We will need the cyclotron frequency more often. Please write a function which takes the magnetic field as first parameter, the "atomic number" of the particle (-1 for an electron, +1 for a proton, +2 for a Helium nucleus and so on) as 2nd parameter with -1 as default, and the mass as third parameter with the electron mass as default. Return the cyclotron frequency in Hz. Make sure that it is never negative!

**[3 marks]**

Let's test it:

## Submission

Before you submit your work you should make a few checks that everything works fine.

1. Save your notebook as a PDF (File->Download As->PDF). This document will help you debugging in the next step.
1. If PDF export does not work: You can do File->Print Preview and then print to a file.
1. Restart the kernel and rerun the entire notebook (Kernel->Restart & Run All). This will delete all variables (but not your code) and rerun the notebook in one go. If this does not go through the endthen you have to fix it. You will see at which cell the run stopped. A common mistake is using a variable that is defined only at a later stage.
1. You think you fixed everything? Redo step 2 (Kernel->Restart & Run All)

You have to download and submit 2 files, the jupyter notebook and a pdf.
- Jupyter notebook. File->Download As->Notebook (.ipynb). Save this file on your disk.
- PDF file. File->Download As->PDF. Save this file on your disk.
- If PDF export does not work. You can do File->Print Preview and then print to a file.

Please submit the two files on Ulwazi.